In [1]:
from pyhive import hive

# 1. Kết nối đến HiveServer2
conn = hive.Connection(host='localhost', port=10000, username='hive', database='default')
cursor = conn.cursor()

# 2. Tạo database nếu chưa có
cursor.execute("CREATE DATABASE IF NOT EXISTS cancer_db")
cursor.execute("USE cancer_db")

# 3. Xóa bảng nếu đã tồn tại
cursor.execute("DROP TABLE IF EXISTS cancer_table")
cursor.execute("DROP TABLE IF EXISTS cancer_table_stage")  # Xóa bảng tạm nếu có

# 4. Tạo bảng chính hỗ trợ transactional (PHẢI lưu dưới dạng ORC, bucketed và có thuộc tính transactional)
cursor.execute("""
  CREATE TABLE cancer_table (
    patient_id STRING,
    age INT,
    gender INT,
    air_pollution INT,
    alcohol_use INT,
    dust_allergy INT,
    occupational_hazards INT,
    genetic_risk INT,
    chronic_lung_disease INT,
    balanced_diet INT,
    obesity INT,
    smoking INT,
    passive_smoker INT,
    chest_pain INT,
    coughing_of_blood INT,
    fatigue INT,
    weight_loss INT,
    shortness_of_breath INT,
    wheezing INT,
    swallowing_difficulty INT,
    clubbing_of_finger_nails INT,
    frequent_cold INT,
    dry_cough INT,
    snoring INT,
    level STRING
  )
  CLUSTERED BY (patient_id) INTO 4 BUCKETS
  STORED AS ORC
  TBLPROPERTIES (
    "transactional"="true"
  )
""")

# 5. Tạo bảng tạm để tải dữ liệu từ CSV (bảng này dùng TEXTFILE để LOAD DATA được)
cursor.execute("""
  CREATE TABLE cancer_table_stage (
    patient_id STRING,
    age INT,
    gender INT,
    air_pollution INT,
    alcohol_use INT,
    dust_allergy INT,
    occupational_hazards INT,
    genetic_risk INT,
    chronic_lung_disease INT,
    balanced_diet INT,
    obesity INT,
    smoking INT,
    passive_smoker INT,
    chest_pain INT,
    coughing_of_blood INT,
    fatigue INT,
    weight_loss INT,
    shortness_of_breath INT,
    wheezing INT,
    swallowing_difficulty INT,
    clubbing_of_finger_nails INT,
    frequent_cold INT,
    dry_cough INT,
    snoring INT,
    level STRING
  )
  ROW FORMAT DELIMITED
  FIELDS TERMINATED BY ','
  STORED AS TEXTFILE
  TBLPROPERTIES ("skip.header.line.count"="1")
""")

# 6. Load dữ liệu vào bảng tạm
cursor.execute("""
  LOAD DATA LOCAL INPATH '/data/Cancer.csv'
  OVERWRITE INTO TABLE cancer_table_stage
""")
print("✅ CSV loaded into cancer_table_stage.")

# 7. Thiết lập các tham số ACID để cho phép chèn dữ liệu vào bảng transactional.
cursor.execute("SET hive.support.concurrency=true")
cursor.execute("SET hive.txn.manager=org.apache.hadoop.hive.ql.lockmgr.DbTxnManager")
cursor.execute("SET hive.compactor.initiator.on=true")
cursor.execute("SET hive.compactor.worker.threads=1")

# 8. Chuyển dữ liệu từ bảng tạm vào bảng chính (dùng INSERT INTO SELECT)
cursor.execute("""
  INSERT INTO cancer_table SELECT * FROM cancer_table_stage
""")
print("✅ Data inserted into cancer_table.")

# 9. Kiểm tra dữ liệu
cursor.execute("SELECT COUNT(*) FROM cancer_table")
row_count = cursor.fetchall()
print(f"📊 Total rows in cancer_table: {row_count[0][0]}")

# 10. Đóng kết nối
cursor.close()
conn.close()


✅ CSV loaded into cancer_table_stage.
✅ Data inserted into cancer_table.
📊 Total rows in cancer_table: 1000


In [3]:
from pyhive import hive

# 1. Kết nối đến HiveServer2 và chọn database.
conn = hive.Connection(host='localhost', port=10000, username='hive', database='cancer_db')
cursor = conn.cursor()

# 3. Chèn dòng dữ liệu mới vào bảng transactional.
# Chú ý: Bảng có 25 cột, nên ta phải cung cấp đúng 25 giá trị theo thứ tự:
# patient_id, age, gender, air_pollution, alcohol_use, dust_allergy, occupational_hazards, genetic_risk,
# chronic_lung_disease, balanced_diet, obesity, smoking, passive_smoker, chest_pain, coughing_of_blood,
# fatigue, weight_loss, shortness_of_breath, wheezing, swallowing_difficulty, clubbing_of_finger_nails,
# frequent_cold, dry_cough, snoring, level

insert_new_query = """
INSERT INTO cancer_table VALUES
('P2000', 50, 1, 4, 1, 2, 3, 1, 1, 2, 3, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 'Medium')
"""

try:
    cursor.execute(insert_new_query)
    conn.commit()
    print("✅ New row inserted successfully!")
except Exception as e:
    print("❌ Error inserting new row:", e)

# 4. Kiểm tra xem dòng mới đã được chèn vào hay chưa.
cursor.execute("SELECT * FROM cancer_table WHERE patient_id = 'P2001'")
new_row = cursor.fetchall()
print("New row:")
print(new_row)


# Đóng kết nối.
# cursor.close()
# conn.close()


✅ New row inserted successfully!
New row:
[('P2000', 50, 1, 4, 1, 2, 3, 1, 1, 2, 3, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 'Medium')]


In [2]:
from pyhive import hive

# Kết nối đến Hive
conn = hive.Connection(host='localhost', port=10000, username='hive', database='cancer_db')
cursor = conn.cursor()

cursor.execute("SET hive.support.concurrency")
print(cursor.fetchall())
cursor.execute("SET hive.txn.manager")
print(cursor.fetchall())
cursor.execute("SET hive.compactor.initiator.on")
print(cursor.fetchall())
cursor.execute("SET hive.compactor.worker.threads")
print(cursor.fetchall())

[('hive.support.concurrency=true',)]
[('hive.txn.manager=org.apache.hadoop.hive.ql.lockmgr.DbTxnManager',)]
[('hive.compactor.initiator.on=true',)]
[('hive.compactor.worker.threads=1',)]


In [11]:
conn = hive.Connection(
    host="localhost",
    port=10000,
    username="hive",
    database="cancer_db"
)
cursor = conn.cursor()
cursor.execute("SELECT * FROM cancer_table WHERE patient_id = 'P2001'")
new_row = cursor.fetchall()
print("New row:")
print(new_row)

New row:
[]


In [5]:
cursor.execute("SELECT COUNT(*) FROM cancer_table")
row_count = cursor.fetchall()
print(f"📊 Total rows in cancer_table: {row_count[0][0]}")

📊 Total rows in cancer_table: 1006


In [9]:
from pyhive import hive

# Kết nối đến HiveServer2
conn = hive.Connection(
    host="localhost",
    port=10000,
    username="hive",
    database="cancer_db"
)
cursor = conn.cursor()

try:
    # Xóa dòng có patient_id là P2001
    delete_sql = "DELETE FROM cancer_table WHERE patient_id = 'P2001'"
    cursor.execute(delete_sql)
    conn.commit()
    print("✅ Row with patient_id='P2001' has been deleted successfully.")
except Exception as e:
    print(f"❌ Lỗi khi xóa dữ liệu: {e}")

# Đóng kết nối (tùy chọn)
cursor.close()
conn.close()


✅ Row with patient_id='P2001' has been deleted successfully.
